In [ ]:
!pip install -q -U google-genai
# Remember to add your gemini api key using the secrets menu on the left

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.3/229.3 kB 4.5 MB/s eta 0:00:00


In [ ]:
from typing import Any, List, Tuple, Callable
import sys
sys.set_int_max_str_digits(0)

# ------------------- Database -------------------

class Database:
    def __init__(self):
        self.results = []  # main generated programs
        self.inspirations = []  # new ideas to try
        self._next_result_id = 1
        self._next_inspiration_id = 1

    def add_result(self, program, results, base_prompt):
        result_entry = {
            "id": self._next_result_id,
            "program": program,
            "results": results,
            "base_prompt": base_prompt
        }
        self.results.append(result_entry)
        self._next_result_id += 1
        return result_entry["id"]

    def add_inspiration(self, parent_result_id, description, result_id=None):
        inspiration_entry = {
            "id": self._next_inspiration_id,
            "parent_result_id": parent_result_id,
            "description": description,
            "result_id": result_id
        }
        self.inspirations.append(inspiration_entry)
        self._next_inspiration_id += 1
        return inspiration_entry["id"]

    def sample(self) -> Tuple[Any, List[Any], str]:
        """
        Returns:
        - parent program (best result program)
        - inspirations (ideas associated with the parent that have no result yet)
        - base_prompt (from parent program)
        """
        if not self.results:
            parent_program = "initial_program"
            inspirations = []
            base_prompt = "Initial base prompt."
            return parent_program, inspirations, base_prompt

        # Find best parent program by score
        best_entry = self.best()
        parent_program = best_entry["program"]
        base_prompt = best_entry["base_prompt"]

        # Get all inspirations for this parent that don't yet have a generated result
        inspirations = [
            insp
            for insp in self.inspirations
            if insp["parent_result_id"] == best_entry["id"] and insp["result_id"] is None
        ]

        return parent_program, inspirations, base_prompt

    def best(self):
      if not self.results:
          return None
      return max(
          self.results,
          key=lambda r: (r["results"].get("score", 0), r["id"])  # prioritize score, then recency to break ties
      )

    def mark_inspiration_as_used(self, inspiration: Any, result_id: int):
        """
        Updates the inspiration that matches `inspiration_description` and has no result_id yet,
        setting its `result_id` to the given result_id.
        """
        for insp_entry in self.inspirations:
            if insp_entry["id"] == inspiration["id"] and insp_entry["result_id"] is None:
                insp_entry["result_id"] = result_id

# ------------------- Prompt Sampler -------------------

class PromptSampler:
    def __init__(self, llm):
        self.llm = llm  # LLM instance used to generate new base prompt

    def build(self, parent_program: str, base_prompt: str, inspiration: str) -> str:
        """
        Use the LLM to generate a new base prompt given the previous base prompt,
        the parent program, and an inspiration. Returns the combined prompt for diff generation.
        """
        if inspiration:
            new_base_prompt = self.llm.generate_new_base_prompt(
                base_prompt=base_prompt,
                parent_program=parent_program,
                inspiration=inspiration
            )
        else:
            new_base_prompt = base_prompt

        # Combine into full prompt for diff generation
        combined_prompt = f"""
Base Prompt for this iteration:
{new_base_prompt}

Parent Program:
{parent_program}

Instructions:
Generate diffs to improve the parent program. Use the following format for all changes:

<<<<<<< SEARCH
# Original code block to be found and replaced
=======
# New code block to replace the original
>>>>>>> REPLACE

Make sure the final computed value to be evaluated is assigned to the variable `result`.
"""
        return combined_prompt, new_base_prompt

# ------------------- LLM -------------------

from google import genai

class LLM:
    def __init__(self):
        self.client = genai.Client() # Gets API key from GEMINI_API_KEY

    def generate(self, base_prompt):
        response = self.client.models.generate_content(
            model="gemini-2.5-flash",
            contents=base_prompt
        )
        diff_text = response.candidates[0].content.parts[0].text
        return diff_text, base_prompt

    def apply_diff(self, parent_program, diff):
        import re
        pattern = re.compile(
            r"<<<<<<< SEARCH\n(.*?)\n=======\n(.*?)\n>>>>>>> REPLACE",
            re.DOTALL
        )

        updated_program = parent_program
        for match in pattern.finditer(diff):
            original_block = match.group(1).strip("\n")
            new_block = match.group(2).strip("\n")
            updated_program = updated_program.replace(original_block, new_block)
        return updated_program

    def generate_inspiration_regression(
        self,
        parent_base_prompt: str,
        parent_program: str,
        parent_results: dict,
        child_program: str,
        child_results: dict
    ) -> str:
        """
        Produce a short inspiration idea when a new child performed worse than the parent.
        The idea should hypothesize about the hidden evaluation criteria and suggest
        prompt updates that could improve score next time.
        """
        prompt = f"""
You are analyzing a code evolution experiment where the goal is to maximize an UNKNOWN (hidden) score.

CONTEXT (Parent performed BETTER than Child):
- Parent Base Prompt:
{parent_base_prompt}

- Parent Program:
{parent_program}

- Parent Results (incl. score):
{parent_results}

- Child Program (performed worse):
{child_program}

- Child Results (incl. score):
{child_results}

TASK:
1) Diagnose why the Child likely underperformed relative to the Parent.
2) Hypothesize about the hidden scoring function (what it may reward/penalize).
3) Recommend 1–3 concrete, testable updates to the BASE PROMPT that better target the hidden criteria.
4) Summarize the above as ONE concise "inspiration idea" that we can try next iteration.

OUTPUT FORMAT:
- Return ONLY a brief inspiration idea (1–3 sentences), no lists or extra formatting.
- The idea should mention the suspected scoring factors and how to update the base prompt accordingly.
"""
        response = self.client.models.generate_content(
            model="gemini-2.5-flash",
            contents=prompt
        )
        return response.candidates[0].content.parts[0].text


    def generate_new_base_prompt(self, base_prompt: str, parent_program: str, inspiration: str) -> str:
        """
        Call Gemini to create a new base prompt given previous prompt, program, and inspiration.
        """
        prompt = f"""
You are improving a code generation prompt.

Previous Base Prompt:
{base_prompt}

Parent Program:
{parent_program}

Inspiration Idea:
{inspiration["description"]}

Generate a new base prompt that incorporates the inspiration idea
and would lead to an improved program.
Return only the new prompt as plain text.
"""
        response = self.client.models.generate_content(
            model="gemini-2.5-flash",
            contents=prompt
        )
        new_prompt = response.candidates[0].content.parts[0].text
        return new_prompt

    def generate_recommendation(self, base_prompt, program, results):
        """
        Ask Gemini to suggest future improvements with explicit reasoning
        about the UNKNOWN scoring function.
        """
        prompt = f"""
You are reviewing a code generation experiment where the evaluation score is HIDDEN/UNKNOWN.
Assume we want to maximize that hidden score and learn from this iteration.

Base Prompt:
{base_prompt}

Generated Program:
{program}

Evaluation Results (incl. score):
{results}

TASK:
1) Infer what the hidden scoring function might reward or penalize based on the current outcome.
2) Propose 1–3 concise, testable updates to the BASE PROMPT that exploit those hypotheses.
3) Suggest one short "inspiration idea" (1–3 sentences) to try next iteration.

OUTPUT:
Return only the short inspiration idea (1–3 sentences), no extra formatting.
"""
        response = self.client.models.generate_content(
            model="gemini-2.5-flash",
            contents=prompt
        )
        return response.candidates[0].content.parts[0].text

# ------------------- Evaluator -------------------

class Evaluator:
    def __init__(self, eval_fn: Callable[[str, Any], dict]):
        self.eval_fn = eval_fn

    def execute(self, program: str) -> dict:
        local_env = {}
        try:
            exec(program, {}, local_env)
            result_value = local_env.get("result", None)
        except Exception as e:
            return {"score": -1, "error": str(e)}

        return self.eval_fn(program, result_value)

# ------------------- Example Custom Eval -------------------

def my_custom_eval(program: str, result: Any) -> dict:
    score = result if isinstance(result, int) else 0
    return {"score": score, "output": f"Custom evaluated {program}"}

# ------------------- Enabler -------------------

class Enabler:
    def __init__(self, database, prompt_sampler, llm, evaluator, iterations=20):
        self.database = database
        self.prompt_sampler = prompt_sampler
        self.llm = llm
        self.evaluator = evaluator
        self.iterations = iterations

    def run(self):
        for i in range(self.iterations):
            print(f"\n=== Iteration {i+1}/{self.iterations} ===")

            # Sample the current best parent and its unused inspirations
            parent_program, inspirations, base_prompt = self.database.sample()

            # Capture the parent entry & score at the start of the iteration
            parent_entry = self.database.best()
            parent_score = parent_entry["results"].get("score", 0)

            # Try each available inspiration
            for inspiration in inspirations:
                # Build combined prompt (returns new base prompt as well)
                combined_prompt, new_base_prompt = self.prompt_sampler.build(
                    parent_program, base_prompt, inspiration
                )

                # Get LLM diff, apply it, evaluate the child
                diff, _ = self.llm.generate(combined_prompt)
                child_program = self.llm.apply_diff(parent_program, diff)
                results = self.evaluator.execute(child_program)
                child_score = results.get("score", 0)

                # Save the child result
                result_id = self.database.add_result(child_program, results, new_base_prompt)

                # If we used an inspiration, mark it as used now
                self.database.mark_inspiration_as_used(inspiration=inspiration, result_id=result_id)


                print(f"\nNew Prompt:\n{new_base_prompt}")
                print(f"\nChild Program:\n{child_program}")
                print(f"Evaluation Results:\n{results}")


                # === NEW: If the child regressed vs parent, generate an inspiration for the PARENT ===
                if child_score < parent_score:
                    regression_insp = self.llm.generate_inspiration_regression(
                        parent_base_prompt=base_prompt,
                        parent_program=parent_program,
                        parent_results=parent_entry["results"],
                        child_program=child_program,
                        child_results=results
                    )
                    # Attach this inspiration to the parent result id (NOT the child)
                    self.database.add_inspiration(
                        parent_result_id=parent_entry["id"],
                        description=regression_insp
                    )
                    print("\nRegression Inspiration (for Parent):\n", regression_insp)
                else:
                    # Otherwise, add a normal inspiration based on the child
                    forward_insp = self.llm.generate_recommendation(
                        new_base_prompt,
                        child_program,
                        results
                    )
                    self.database.add_inspiration(
                        parent_result_id=result_id,
                        description=forward_insp
                    )
                    print("\nForward Inspiration (for Child):\n", forward_insp)

            # Show current best
            best_entry = self.database.best()
            print("\nBest so far:", best_entry)

        # Final best output
        best_entry = self.database.best()
        print("Best Evaluation Results:\n", best_entry)

# ------------------- Example Usage -------------------

# Seed database
database = Database()
initial_program = """
def compute():
    return 10

result = compute()
"""
initial_base_prompt = """
This prompt will be used to generate code, but it is unclear exactly what the evaluation
metric is. You should write code that you think would maximize some kind of score. Keep the code relatively short and simple.
Focus on producing code that is correct, well-structured, and likely to achieve a high score.
"""

database.add_result(initial_program, my_custom_eval(initial_program, 10), initial_base_prompt)
database.add_inspiration(parent_result_id=1, description="")

# Initialize components
llm = LLM()
prompt_sampler = PromptSampler(llm)
evaluator = Evaluator(eval_fn=my_custom_eval)
enabler = Enabler(database, prompt_sampler, llm, evaluator, iterations=3)

# Run the evolution
enabler.run()


ValueError: Missing key inputs argument! To use the Google AI API, provide (`api_key`) arguments. To use the Google Cloud API, provide (`vertexai`, `project` & `location`) arguments.

In [ ]:
# Experiment explicitly stating reward function
# Seed database
database = Database()
initial_program = """
def compute():
    return 10

result = compute()
"""
initial_base_prompt = """
This prompt will be used to generate code, and the evaluation metric is the size of the number set to result at the end of the function. You should write code that you think would maximize this score.
Focus on producing code that is correct, well-structured, and likely to achieve a high score.
"""

database.add_result(initial_program, my_custom_eval(initial_program, 10), initial_base_prompt)
database.add_inspiration(parent_result_id=1, description="")

# Initialize components
llm = LLM()
prompt_sampler = PromptSampler(llm)
evaluator = Evaluator(eval_fn=my_custom_eval)
enabler = Enabler(database, prompt_sampler, llm, evaluator, iterations=3)

# Run the evolution
enabler.run()


=== Iteration 1/3 ===
Parent Program:
 
def compute():
    return 10

result = compute()

Remaining Inspirations:
 ['']

Child Program:

def compute():
    # Maximize the size of the number set returned.
    # The evaluation metric is the size of the number set assigned to 'result'.
    # By returning a large Python set, we directly provide a collection
    # whose length corresponds to the desired score.
    # The actual maximum size is limited by available memory.
    # 10 million unique integers is a substantial number that should
    # be achievable on most modern systems without excessive memory strain.
    num_elements = 10_000_000  # Ten million unique numbers
    return set(range(num_elements))

result = compute()

Evaluation Results:
{'score': 0, 'output': "Custom evaluated \ndef compute():\n    # Maximize the size of the number set returned.\n    # The evaluation metric is the size of the number set assigned to 'result'.\n    # By returning a large Python set, we directly pr

In [ ]:
# example from the alphazero blog post https://deepmind.google/discover/blog/alphaevolve-a-gemini-powered-coding-agent-for-designing-advanced-algorithms/
class Experiment(experiment.RankExperiment):
    """Rank tensor decomposition optimization. Assume single-device."""

    def __init__(self, mode, init_rng, config, hypers):
        self.hypers = hypers
        super().__init__(mode=mode, init_rng=init_rng, config=config)

    def _get_optimizer(self) -> optax.GradientTransformation:
        """Returns optimizer."""
        return optax.adam(self.hypers.learning_rate)

    def _get_init_fn(self) -> jax.nn.initializers.Initializer:
        """Returns initializer function."""
        scale = self.hypers.init_scale
        return initializers.normal(0 + 1j * 0.1 + 1j * scale, jnp.complex64)

    def _linear_schedule(self, global_step, start: float = 0.0, end: float = 0.0):
        frac = 1 - global_step / self.config.training_steps
        return (start - end) * frac + end

    @functools.partial(jax.jit, static_argnums=0)
    def _update_func(
        self,
        decomposition: tuple[jnp.ndarray, jnp.ndarray, jnp.ndarray],
        opt_state: optax.OptState,
        global_step: jnp.ndarray,
        rng: jnp.ndarray,
    ) -> tuple[
        tuple[jnp.ndarray, jnp.ndarray, jnp.ndarray],
        optax.OptState,
        jnp.ndarray,
    ]:
        """A single step of decomposition parameter updates."""
        # Compute loss and gradients.
        loss, grads = jax.value_and_grad(
            lambda decomposition, global_step, rng: jnp.mean(
                self._loss_fn(decomposition, global_step, rng)
            )
        )(decomposition, global_step, rng)
        # When optimizing real-valued functions of complex variables, we must take
        # the conjugate of the gradient.
        grads = jax.tree_util.tree_map(lambda x: x.conj(), grads)
        # Gradient updates.
        updates, opt_state = self.opt.update(grads, opt_state, decomposition)
        decomposition = optax.apply_updates(decomposition, updates)
        return decomposition, opt_state, loss

    def _loss_fn(
        self,
        decomposition: tuple[jnp.ndarray, jnp.ndarray, jnp.ndarray],
        global_step,
        rng: chex.PRNGKey,
    ) -> jnp.ndarray:
        """Computes (batched) loss on learned decomposition."""
        # Compute reconstruction loss.
        rec_tensor = self._decomposition_to_tensor(decomposition)  # (B, N, N, R)
        # Add a batch dimension to `target_tensor` to ensure correct broadcasting.
        # Define the loss as the L2 reconstruction error.
        rec_loss = l2_loss_complex(self.target_tensor[None, ...], rec_tensor)

        # We must return a real-valued loss.
        return jnp.real(rec_loss)


def l2_loss_complex(x: jnp.ndarray, y: jnp.ndarray) -> jnp.ndarray:
    """Elementwise L2 loss for complex numbers."""
    # While product will have imaginary part zero, use jnp.real to get a float
    # array (instead of an array with entries of the type x + 0j).
    return jnp.real((x - y).conj() * (x - y))


def sweep():
    """Returns a sweep over hyperparameters.

    Syntax for uniform hyperparameters:
    hyper.uniform('name', hyper.interval(min, max))
    Syntax for discrete hyperparameters:
    hyper.uniform('name', hyper.discrete([value1, value2, ...]))
    """

    return hyper.zipit([
        hyper.uniform('init_scale', hyper.interval(0.2, 1.5)),
        hyper.uniform('learning_rate', hyper.interval(0.05, 0.3)),
    ])


In [ ]:
# Tensor verifier from the colab https://colab.research.google.com/github/google-deepmind/alphaevolve_results/blob/master/mathematical_results.ipynb#scrollTo=JCmE5JpLeJ2B
#@title Verification function
import numpy as np

def verify_tensor_decomposition(decomposition: tuple[np.ndarray, np.ndarray, np.ndarray], n: int, m: int, p: int, rank: int):
  """Verifies the correctness of the tensor decomposition.

  Args:
    decomposition: a tuple of 3 factor matrices with the same number of columns.
      (The number of columns specifies the rank of the decomposition.) To
      construct a tensor, we take the outer product of the i-th column of the
      three factor matrices, for 1 <= i <= rank, and add up all these outer
      products.
    n: the first parameter of the matrix multiplication tensor.
    m: the second parameter of the matrix multiplication tensor.
    p: the third parameter of the matrix multiplication tensor.
    rank: the expected rank of the decomposition.

  Raises:
    AssertionError: If the decomposition does not have the correct rank, or if
    the decomposition does not construct the 3D tensor which corresponds to
    multiplying an n x m matrix by an m x p matrix.
  """
  # Check that each factor matrix has the correct shape.
  factor_matrix_1, factor_matrix_2, factor_matrix_3 = decomposition
  assert factor_matrix_1.shape == (n * m, rank), f'Expected shape of factor matrix 1 is {(n * m, rank)}. Actual shape is {factor_matrix_1.shape}.'
  assert factor_matrix_2.shape == (m * p, rank), f'Expected shape of factor matrix 1 is {(m * p, rank)}. Actual shape is {factor_matrix_2.shape}.'
  assert factor_matrix_3.shape == (p * n, rank), f'Expected shape of factor matrix 1 is {(p * n, rank)}. Actual shape is {factor_matrix_3.shape}.'

  # Form the matrix multiplication tensor <n, m, p>.
  matmul_tensor = np.zeros((n * m, m * p, p * n), dtype=np.int32)
  for i in range(n):
    for j in range(m):
      for k in range(p):
        matmul_tensor[i * m + j][j * p + k][k * n + i] = 1

  # Check that the tensor is correctly constructed.
  constructed_tensor = np.einsum('il,jl,kl -> ijk', *decomposition)
  assert np.array_equal(constructed_tensor, matmul_tensor), f'Tensor constructed by decomposition does not match the matrix multiplication tensor <{(n,m,p)}>: {constructed_tensor}.'
  print(f'Verified a decomposition of rank {rank} for matrix multiplication tensor <{n},{m},{p}>.')

  # Print the set of values used in the decomposition.
  np.set_printoptions(linewidth=100)
  print('This decomposition uses these factor entries:\n', np.array2string(np.unique(np.vstack((factor_matrix_1, factor_matrix_2, factor_matrix_3))), separator=', '))